In [ ]:
# Function to scrape ISBN from Google Books page based on provided HTML structure

import pandas as pd
df = pd.read_csv('books_data.csv')

df['previewLink'] = df['previewLink'].fillna('')

def shorten_google_books_link(full_link):
    # Find the index of the '&' character to get the substring before it
    index = full_link.find('&')
    if index != -1:  # If '&' character is found
        shortened_link = full_link[:index]
    else:  # If '&' character is not found, return the original link
        shortened_link = full_link
    return shortened_link

# Apply shorten_google_books_link function to create a new column with shortened links
df['ShortenedLink'] = df['previewLink'].apply(shorten_google_books_link)






In [ ]:
import multiprocessing

def process_chunk(chunk):
    # Apply the scraping function to the chunk and add ISBN column
    chunk['ISBN'] = chunk['ShortenedLink'].apply(scrape_isbn)
    return chunk

num_cores = multiprocessing.cpu_count()
pool = multiprocessing.Pool(num_cores)

chunk_size = 250
start_index = 20000
end_index = start_index + chunk_size
output_file = "processed_data_new2.csv"
while start_index < len(df):
    chunk = df.iloc[start_index:end_index]
    processed_chunk = pool.apply(process_chunk, args=(chunk,))
    # Do something with processed_chunk, e.g., save it to a file
    # Inside the loop where chunks are processed
    with open(output_file, 'a') as f:
      processed_chunk.to_csv(f, index=False, header=not start_index)  # Only write header for the first chunk
    print(f"Appended processed chunk with start index: {start_index} to {output_file}")

    # Update start_index and end_index for the next chunk
    start_index = end_index
    end_index = min(start_index + chunk_size, len(df))

#processed_chunk = pool.apply(process_chunk, args=(chunk,))
pool.close()
pool.join()

In [ ]:
file_paths = ['books_with_isbn_chunk_1.csv', 'books_with_isbn_chunk_2.csv', 'books_with_isbn_chunk_3.csv', 
              'books_with_isbn_chunk_4.csv', 'books_with_isbn_chunk_5.csv', 'books_with_isbn_chunk_6.csv', 
              'books_with_isbn_chunk_7.csv', 'books_with_isbn_chunk_8.csv', 'books_with_isbn_chunk_9.csv', 
              'books_with_isbn_chunk_13.csv', 'books_with_isbn_chunk_12.csv', 'books_with_isbn_chunk_11.csv', 'processed_data.csv',
              'processed_data2.csv', 'processed_data4.csv', 'processed_data5.csv', 'processed_data6.csv', 'processed_data7.csv',
              'processed_data8.csv', 'processed_data10.csv', 'processed_data11.csv', 'processed_data12.csv', 'processed_data13.csv', 'processed_data15.csv',
              'processed_data16.csv', 'processed_data17.csv']

# Initialize an empty list to store DataFrames
dfs = []

# Iterate over each CSV file path
for file_path in file_paths:
    # Read the CSV file into a DataFrame and append it to the list
    dfs.append(pd.read_csv(file_path))

# Concatenate all DataFrames in the list
concatenated_df = pd.concat(dfs, ignore_index=True, axis=1)

# Write the concatenated DataFrame to a new CSV file
concatenated_df.to_csv('datawithISBN.csv', index=False)


newdf = pd.read_csv('datawithISBN.csv')
